# Self-Reflection Pattern

In [1]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.agents.utils import nodes_to_message_history

## Turn 1: Initial Solution

In [2]:
agent = get_agent()

prompt_1 = """Write a Python function that validates email addresses.
The function should return True if the email is valid, False otherwise."""

agent_run_1, nodes_1 = await run_agent(agent, prompt_1)

assert agent_run_1 is not None and agent_run_1.result is not None
print("Initial solution:")
print(agent_run_1.result.output)

Initial solution:
# Email Validation Function

Here's a comprehensive email validation function with multiple approaches:

## Basic Version (Regex-based)

```python
import re

def validate_email(email):
    """
    Validates an email address using regex pattern.
    
    Args:
        email (str): Email address to validate
    
    Returns:
        bool: True if valid, False otherwise
    """
    if not email or not isinstance(email, str):
        return False
    
    # Regex pattern for email validation
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    
    return re.match(pattern, email) is not None
```

## More Robust Version

```python
import re

def validate_email_robust(email):
    """
    More comprehensive email validation with additional checks.
    
    Args:
        email (str): Email address to validate
    
    Returns:
        bool: True if valid, False otherwise
    """
    if not email or not isinstance(email, str):
        return False
    
    # R

## Turn 2: Self-Reflection (Critique)

In [3]:
message_history = nodes_to_message_history(nodes_1)

prompt_2 = """Review your solution and identify potential issues:
1. Are there any edge cases not handled?
2. Are there any security concerns?
3. Could the implementation be more robust?

List the specific problems you find."""

agent_run_2, nodes_2 = await run_agent(agent, prompt_2, message_history=message_history)

assert agent_run_2 is not None and agent_run_2.result is not None
print("\nCritique:")
print(agent_run_2.result.output)


Critique:
# Critical Review of Email Validation Solution

## **Identified Problems**

### **1. Edge Cases Not Handled**

#### **a) Special Characters in Local Part**
```python
# Current regex allows invalid patterns:
validate_email_robust("user..name@example.com")  # Has consecutive dots - CAUGHT ✓
validate_email_robust(".user@example.com")       # Starts with dot - NOT CAUGHT ✗
validate_email_robust("user.@example.com")       # Ends with dot - NOT CAUGHT ✗
```

**Problem**: The regex `^[a-zA-Z0-9][a-zA-Z0-9._%+-]*[a-zA-Z0-9]@` requires start/end to be alphanumeric, but emails with only 1 character in local part (like `a@b.co`) would fail this pattern.

#### **b) Quoted Strings**
```python
# Valid per RFC 5321 but not handled:
"user name"@example.com      # Quoted local part - REJECTED ✗
"user@domain"@example.com    # @ symbol in quotes - REJECTED ✗
```

#### **c) IP Address Domains**
```python
# Valid per RFC but not handled:
user@[192.168.1.1]          # IPv4 literal - REJECTED ✗
us

## Turn 3: Revised Solution

In [4]:
message_history = nodes_to_message_history(nodes_2)

prompt_3 = """Based on your critique, provide an improved version of the function that addresses the issues you identified."""

agent_run_3, nodes_3 = await run_agent(agent, prompt_3, message_history=message_history)

assert agent_run_3 is not None and agent_run_3.result is not None
print("\nRevised solution:")
print(agent_run_3.result.output)


Revised solution:
# Improved Email Validation Implementation

Here's a production-ready implementation that addresses all identified issues:

```python
import re
from typing import Tuple, Optional
from dataclasses import dataclass


@dataclass
class EmailValidationResult:
    """Result of email validation with detailed information."""
    is_valid: bool
    normalized_email: Optional[str] = None
    error_message: Optional[str] = None
    error_code: Optional[str] = None


class EmailValidator:
    """
    Robust, secure email validator compliant with RFC 5321/5322 (practical subset).
    
    Features:
    - Protection against ReDoS attacks
    - CRLF injection prevention
    - International domain name (IDN) support
    - Detailed error reporting
    - Email normalization
    - Configurable strictness levels
    """
    
    # Constants
    MAX_EMAIL_LENGTH = 320  # Practical maximum
    MAX_LOCAL_PART_LENGTH = 64
    MAX_DOMAIN_LENGTH = 253
    MAX_LABEL_LENGTH = 63
    MIN_TLD_LEN

## Comparison: Direct Prompting (No Reflection)

What if we simply ask for a "robust" solution directly, without the reflection cycle?

In [5]:
agent_direct = get_agent()

prompt_direct = """Write a robust Python function that validates email addresses.
The function should return True if the email is valid, False otherwise.
Handle edge cases and security concerns."""

agent_run_direct, _ = await run_agent(agent_direct, prompt_direct)

assert agent_run_direct is not None and agent_run_direct.result is not None
print("Direct solution (no reflection):")
print(agent_run_direct.result.output)

Direct solution (no reflection):
```python
import re
from typing import Optional

def validate_email(email: str, check_dns: bool = False) -> bool:
    """
    Robust email validation function with security considerations.
    
    Args:
        email: Email address to validate
        check_dns: Whether to check if domain has MX records (requires dnspython)
    
    Returns:
        True if email is valid, False otherwise
    
    Examples:
        >>> validate_email("user@example.com")
        True
        >>> validate_email("invalid.email")
        False
    """
    
    # Basic security checks
    if not email or not isinstance(email, str):
        return False
    
    # Length validation (RFC 5321)
    if len(email) > 320:  # Max email length
        return False
    
    # Remove leading/trailing whitespace
    email = email.strip()
    
    if not email:
        return False
    
    # Check for common injection attempts
    dangerous_chars = ['\n', '\r', '\0', '<', '>', '(', ')